In [2]:
import numpy as np

In [3]:
# Lowering operator function
def lower_op(d):
    lower_op = np.zeros((d,d))
    for ii in range(0,d-1,1):
        lower_op[ii,ii+1] = np.sqrt(ii+1) 
        
    return lower_op

In [4]:
# Create operators for cavity and transmon
dc = 60
dq = 3

Ic = np.identity(dc)
Iq = np.identity(dq)

lowerc = lower_op(dc)
lowerq = lower_op(dq)

raisec = np.conjugate(np.transpose(lowerc))
raiseq = np.conjugate(np.transpose(lowerq))

In [5]:
# Static Hamiltonian parameters (note the units are GHz)

ωq = 5.1*2*np.pi
ωc = 7.2*2*np.pi
α = -0.35*2*np.pi
g = 0.2*2*np.pi

In [6]:
# Static Hamiltonian

H0 = ωc*np.kron(Iq,raisec@lowerc) + ωq*np.kron(raiseq@lowerq,Ic) + (α/2)*np.kron(raiseq@raiseq@lowerq@lowerq,Ic) + g*(np.kron(lowerq,raisec) + np.kron(raiseq,lowerc))


In [7]:
# Some physics to calculate the right drive frequency for our standard setup

Δ = ωq - ωc
χ01 = g**2/Δ
χ = (g**2/Δ/(Δ+α)*α)

In [8]:
# Pulse shape for the drive

def step_f(t,t_c):
    return 0.5 * (np.sign(t-t_c) + 1)

def gaussian(t,sig, ti, tf):
    return  np.exp(-(t - (tf+ti)/2)**2 / (2 * sig**2))

def gaussian_square(t, rise, Δt):
    return (step_f(t,rise) - step_f(t,Δt-rise)) + (step_f(t,0.0) - step_f(t,rise))*gaussian(t,rise/4,0.,2*rise) + (step_f(t,Δt-rise) - step_f(t,Δt))*gaussian(t+2*rise-Δt,rise/4,0.,2*rise)

In [9]:
# Drive parameters

Ω = 2*np.pi*1e-3
ωd = ωc - (χ01 - χ)
Δt0 = 350.0
rise0 = 8.0

In [10]:
# Drive operator and function

Hd = Ω*np.kron(Iq,raisec+lowerc)

def drivef(t):
    return 2*np.cos(ωd*t)*gaussian_square(t,rise0,Δt0)

In [11]:
# Dissipation

# Lindblad operators
L0 = np.kron(Iq,lowerc)
L1 = np.kron(lowerq,Ic)
L2 = np.kron(raiseq*lowerq,Ic)

# Rates
γ0 = 4*1e-3*2*np.pi
γ1 = 1/(350*1e3)
γ2 = 1/(230*1e3) - γ1/2

In [12]:
# Other parameters

# Initial state
rho_in = np.zeros((dq*dc,dq*dc))
rho_in[0,0] = 1.0

# Simulation time
t0 = 0.0
tf = 700.0

#### The rest of these cells define operators to measure the expectation value of

In [13]:
evs, evecs = np.linalg.eig(H0)

In [14]:
# Now this is where things get a bit tricky. We have to sort the eigenvectors so that the basis transformation 
# matrix respects the ordering of Hilbert spaces we imagine for the dressed states: 
# i.e. dressed transmon \otimes dressed cavity
# To do this we use that the hybridization is weak so we can sort using the bare projectors

# Projectors onto bare transmon states
Pg = np.kron(np.array([[1.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]),Ic)
Pe = np.kron(np.array([[0.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,0.0]]),Ic)
Pf = np.kron(np.array([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,1.0]]),Ic)

g_ol = []
e_ol = []
f_ol = []

for i in range(dc*dq):
    g_ol.append(np.trace(Pg@np.outer(evecs[:,i],evecs[:,i])))
    e_ol.append(np.trace(Pe@np.outer(evecs[:,i],evecs[:,i])))
    f_ol.append(np.trace(Pf@np.outer(evecs[:,i],evecs[:,i])))


# Sort by largest to smallest overlap
g_sort = np.argsort(g_ol).tolist()
g_sort.reverse()
e_sort = np.argsort(e_ol).tolist()
e_sort.reverse()
f_sort = np.argsort(f_ol).tolist()
f_sort.reverse()

# Take the largest overlaps
g_ind = g_sort[0:dc]
e_ind = e_sort[0:dc]
f_ind = f_sort[0:dc]

# Confirm no duplicate indices
temp = g_ind + e_ind + f_ind
print(len(temp) == len(set(temp)))

# Now since each of the individual lists are ordered by decreasing overlap with the transmon projector, they
# should be in reverse order of cavity overlap, but just to be safe let's order them. Upon inspection it turns
# out that they actually are not due to the finite truncation, so let's reorder them.

Nc = np.kron(Iq,raisec@lowerc)

c_ol_g = []
c_ol_e = []
c_ol_f = []
for j in range(dc):
    c_ol_g.append(np.trace(Nc@np.outer(evecs[:,g_ind[j]],evecs[:,g_ind[j]])))
    c_ol_e.append(np.trace(Nc@np.outer(evecs[:,e_ind[j]],evecs[:,e_ind[j]])))
    c_ol_f.append(np.trace(Nc@np.outer(evecs[:,f_ind[j]],evecs[:,f_ind[j]])))
    
# Sort by smallest to largest cavity overlap
gc_sort = np.argsort(c_ol_g).tolist()
ec_sort = np.argsort(c_ol_e).tolist()
fc_sort = np.argsort(c_ol_f).tolist()

g_ind_fullsort = [g_ind[k] for k in gc_sort]
e_ind_fullsort = [e_ind[k] for k in ec_sort]
f_ind_fullsort = [f_ind[k] for k in fc_sort]

# Confirm no duplicate indices again
temp2 = g_ind_fullsort + e_ind_fullsort + f_ind_fullsort
print(len(temp2) == len(set(temp2)))

# Finally we reorder the eigenvector matrix to create the change of basis matrix
V = np.zeros((dq*dc,dq*dc))

for n in temp2:
    V[:,n] = evecs[:,temp2[n]]
    
# For sanity check that V still does what it should and diagonalizes H0
tempM = np.conjugate(np.transpose(V))@H0@V
print(np.max(np.abs(tempM - np.diag(np.diagonal(tempM)))))

True
True
1.4332109975590533e-09


In [15]:
# Transform dressed cavity lowering and raising operator back into lab frame
lowerc_dr = V@np.kron(Iq,lowerc)@np.conjugate(np.transpose(V))
raisec_dr = V@np.kron(Iq,raisec)@np.conjugate(np.transpose(V))

In [16]:
# Things to measure about the dressed cavity

# 1) Photon number operator
Nc_dr = raisec_dr@lowerc_dr

# 2) Cavity quadratures

X_dr = (lowerc_dr + raisec_dr)/np.sqrt(2)
Y_dr = -1j*(lowerc_dr - raisec_dr)/np.sqrt(2)

In [17]:
# Things to measure about the dressed transmon: state populations
# This creates projectors that sum over all the dressed eigenstates that correspond to a given dressed
# transmon state, as opposed to just picking the one with zero dressed cavity excitations

# G-state pop
g_proj = np.zeros((dq*dc,dq*dc))
for m in range(dc):
    g_proj = g_proj + np.outer(evecs[:,g_ind[j]],evecs[:,g_ind[j]])

# E-state pop
e_proj = np.zeros((dq*dc,dq*dc))
for m in range(dc):
    e_proj = e_proj + np.outer(evecs[:,e_ind[j]],evecs[:,e_ind[j]])

# F-state pop
f_proj = np.zeros((dq*dc,dq*dc))
for m in range(dc):
    f_proj = f_proj + np.outer(evecs[:,f_ind[j]],evecs[:,f_ind[j]])

# Using dynamics with JAX

Import JAX, set it to use 64 bit and GPU, and setup dynamics to work with JAX in the background.

In [18]:
from qiskit_dynamics.array import Array

# configure jax to use 64 bit mode
import jax
jax.config.update("jax_enable_x64", True)

# tell JAX we are using CPU
jax.config.update('jax_platform_name', 'gpu')

# set default backend
Array.set_default_backend('jax')

import jax.numpy as jnp

Modify the functions to use JAX and Array.

In [19]:
# Pulse shape for the drive

def step_f(t,t_c):
    return 0.5 * (jnp.sign(t-t_c) + 1)

def gaussian(t,sig, ti, tf):
    return  jnp.exp(-(t - (tf+ti)/2)**2 / (2 * sig**2))

def gaussian_square(t, rise, Δt):
    t = Array(t).data
    rise = Array(rise).data
    Δt = Array(Δt).data
    out = (step_f(t,rise) - step_f(t,Δt-rise)) + (step_f(t,0.0) - step_f(t,rise))*gaussian(t,rise/4,0.,2*rise) + (step_f(t,Δt-rise) - step_f(t,Δt))*gaussian(t+2*rise-Δt,rise/4,0.,2*rise)
    return Array(out)

For now just run a simulation without dressing everything to demo how dynamics works.

In [20]:
# Drive parameters

Ω = 2*np.pi*1e-3
ωd = ωc - (χ01 - χ)
Δt0 = 350.0
rise0 = 8.0

# Hamiltonian operators
H0 = H0
Hd = Hd

# Lindblad operators
L0 = np.kron(Iq,lowerc)
L1 = np.kron(lowerq,Ic)
L2 = np.kron(raiseq*lowerq,Ic)

# Rates
γ0 = 4*1e-3*2*np.pi
γ1 = 1/(350*1e3)
γ2 = 1/(230*1e3) - γ1/2

In [21]:
from qiskit_dynamics import Solver, Signal

# setup simulation in the rotating frame of the drift
solver = Solver(
    static_hamiltonian=H0,
    hamiltonian_operators=[Hd],
    static_dissipators=[np.sqrt(γ0) * L0, np.sqrt(γ1) * L1, np.sqrt(γ2) * L2],
    rotating_frame=H0,
    rwa_cutoff=1.1 * ωd
)

In [22]:
# setup initial state
y0 = np.zeros(dc * dq, dtype=complex)
y0[0] = 1.
y0 = np.diag(y0)

# construct a parameterized simulation
def sim_function(ωd, rise0, Δt0):
    
    # drive envelope
    def drivef(t):
        return 2 * gaussian_square(t,rise0,Δt0)
    
    # carrier freqs in Hz
    signal = Signal(envelope=drivef, carrier_freq=ωd / (2 * np.pi))
    
    solver_copy = solver.copy()
    # set list of time-dependent coefficients for Hamiltonian and Lindblad terms
    # in this case there are no time-dependent lindblad terms
    solver_copy.signals=([signal], None)
    
    results = solver_copy.solve(
        t_span=[0, Δt0], 
        y0=y0, 
        method='jax_odeint', 
        atol=1e-10, rtol=1e-10
    )
    
    return results.y[-1]

In [77]:
rise_range = np.arange(7.6, 8.38, .05)
# input_params = jnp.array()
# ω
# rng = np.random.default_rng(123)

# num_inputs=5
# input_params = jnp.array(rng.uniform(low=-0.3, high=0.3, size=(num_inputs, 3)))
ls = ([[ωd, rise0+r, Δt0] for r in rise_range])
input_params = jnp.array(ls)


In [78]:
rng = np.random.default_rng(123)
input_params1 = jnp.array(rng.uniform(low=-0.3, high=0.3, size=(10, 3)))
input_params1.shape
input_params.shape

(16, 3)

In [79]:
from jax import jit, vmap

l_sim_function = lambda params: sim_function(params[0], params[1], params[2])

vmap_sim_func = jit(vmap(l_sim_function))



In [80]:
%%time
vmap_sim_func(input_params).block_until_ready()


CPU times: user 1min 27s, sys: 83.5 ms, total: 1min 27s
Wall time: 1min 26s


DeviceArray([[[ 9.31007088e-01+1.01492253e-18j,
               -2.43032816e-01-4.83450838e-02j,
                4.22432329e-02+2.01031279e-02j, ...,
               -5.05565374e-16+5.41072817e-17j,
               -3.58677908e-17-2.72975770e-17j,
               -3.39736160e-21-4.62067791e-21j],
              [-2.43032816e-01+4.83450838e-02j,
                6.59525690e-02+4.14575421e-18j,
               -1.20712525e-02-3.05419600e-03j, ...,
                1.29162981e-16-4.03692546e-17j,
                1.07807582e-17+5.26382147e-18j,
                1.12688424e-21+1.02985967e-21j],
              [ 4.22432329e-02-2.01031279e-02j,
               -1.20712525e-02+3.05419600e-03j,
                2.35083444e-03+3.26351321e-19j, ...,
               -2.17711462e-17+1.33685957e-17j,
               -2.21701959e-18-4.64266495e-19j,
               -2.53963438e-22-1.36320488e-22j],
              ...,
              [-5.05565374e-16-5.41072817e-17j,
                1.29162981e-16+4.03692546e-17j,
   

In [67]:
# %%time
# vmap_sim_func(input_params).block_until_ready()

In [ ]:
# Time for vmap over 8 is 45 (vs 43 post jit) seconds (5.6 sec for sim)
# Time for vmap over 16 is 86 seconds (83 post jit)((5.38 sec for sim)

In [23]:
%%time
yf = sim_function(ωd, rise0, Δt0).block_until_ready()

CPU times: user 15.5 s, sys: 50.8 ms, total: 15.6 s
Wall time: 15 s


In [24]:
# setup sparse simulation, keep rotating_frame diagonal to preserve sparsity
sparse_solver = Solver(
    static_hamiltonian=H0,
    hamiltonian_operators=[Hd],
    static_dissipators=[np.sqrt(γ0) * L0, np.sqrt(γ1) * L1, np.sqrt(γ2) * L2],
    rotating_frame=np.diag(H0),
    evaluation_mode='sparse'
)

/u/brosand/danDynamics/qiskit-dynamics/qiskit_dynamics/models/lindblad_model.py:150: UserWarning: Using sparse mode with JAX is primarily recommended for use on CPU.
  self._operator_collection = construct_lindblad_operator_collection(
/u/brosand/danDynamics/qiskit-dynamics/qiskit_dynamics/models/lindblad_model.py:500: UserWarning: Using sparse mode with JAX is primarily recommended for use on CPU.
  self._operator_collection = construct_lindblad_operator_collection(


In [25]:
# setup initial state
y0 = np.zeros(dc * dq, dtype=complex)
y0[0] = 1.
y0 = np.diag(y0)

# construct a parameterized simulation
def sparse_sim_function(ωd, rise0, Δt0):
    
    # drive envelope
    def drivef(t):
        return 2 * gaussian_square(t,rise0,Δt0)
    
    # carrier freqs in Hz
    signal = Signal(envelope=drivef, carrier_freq=ωd / (2 * np.pi))
    
    solver_copy = sparse_solver.copy()
    # set list of time-dependent coefficients for Hamiltonian and Lindblad terms
    # in this case there are no time-dependent lindblad terms
    solver_copy.signals=([signal], None)
    
    results = solver_copy.solve(
        t_span=[0, Δt0], 
        y0=y0, 
        method='jax_odeint', 
        atol=1e-10, rtol=1e-10
    )
    
    return results.y[-1]

In [26]:
%%time
yf_sparse = sparse_sim_function(ωd, rise0, Δt0).block_until_ready()

CPU times: user 16.8 s, sys: 39.4 ms, total: 16.8 s
Wall time: 15.7 s


Verify consistency. Since both were simulated in different frames, we first need to map them to the same frame, in this case we choose the lab frame.

In [27]:
# verify consistency
yf_lab = solver.model.rotating_frame.operator_out_of_frame(Δt0, yf)
yf_sparse_lab = sparse_solver.model.rotating_frame.operator_out_of_frame(Δt0, yf_sparse)
np.linalg.norm(yf_lab - yf_sparse_lab) / np.sqrt(dq * dc)

Array(4.34575681e-08)

Can also check a more "physically motivated" measure:

$$ f(\rho_1, \rho_2) = Tr\left(\sqrt{\sqrt{\rho_1}\rho_2\sqrt{\rho_1}}\right)$$

In [28]:
from scipy.linalg import sqrtm

def fidelity(rho1, rho2):
    sqrt_rho1 = sqrtm(rho1)
    return sqrtm(sqrt_rho1 @ rho2 @ sqrt_rho1).trace()

In [29]:
fidelity(yf_lab, yf_sparse_lab)

(1.0000000110167597+5.681207662696184e-10j)